In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# ฟังก์ชันดึงข้อมูลราคาและ Market Cap
def get_historical_data(coin_id, start_date, end_date):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    params = {
        'vs_currency': 'usd',
        'from': int(start_date.timestamp()),
        'to': int(end_date.timestamp()),
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        prices = data['prices']
        market_caps = data['market_caps']
        
        df = pd.DataFrame({
            'date': [datetime.utcfromtimestamp(p[0] / 1000).strftime('%Y-%m-%d') for p in prices],
            'price': [p[1] for p in prices],
            'market_cap': [m[1] for m in market_caps]
        })
        return df
    else:
        print(f"Error fetching data for {coin_id}: {response.status_code}")
        return None

# ดึงรายชื่อ Top 100 เหรียญ
def get_top_100_coins():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {'vs_currency': 'usd', 'order': 'market_cap_desc', 'per_page': 100, 'page': 1}
    response = requests.get(url, params=params)
    return [coin['id'] for coin in response.json()] if response.status_code == 200 else []

# กำหนดช่วงเวลา
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 3, 1)

# ดึงข้อมูลของ Top 100 เหรียญ
top_100_coins = get_top_100_coins()
all_data = {}

for coin in top_100_coins:
    print(f"Fetching data for {coin}...")
    df = get_historical_data(coin, start_date, end_date)
    if df is not None:
        all_data[coin] = df

# บันทึกเป็น CSV
for coin, df in all_data.items():
    df.to_csv(f"{coin}_historical_data.csv", index=False)


In [ ]:
import requests
import pandas as pd
import time

# ฟังก์ชันดึง Top 100 Coins ในแต่ละวัน
def get_top_100(date):
    url = f"https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1,
        "sparkline": "false",
        "price_change_percentage": "24h"
    }
    response = requests.get(url, params=params)
    data = response.json()
    top_100 = [
        {
            "date": date,
            "id": coin["id"],
            "symbol": coin["symbol"],
            "name": coin["name"],
            "market_cap": coin["market_cap"],
            "price": coin["current_price"],
            "rank": idx + 1
        }
        for idx, coin in enumerate(data)
    ]
    return top_100

# เก็บข้อมูลย้อนหลังตั้งแต่ 2020 - 2025
start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2025-01-01")

all_data = []

while start_date < end_date:
    print(f"Fetching data for {start_date.date()} ...")
    try:
        daily_data = get_top_100(start_date.date())
        all_data.extend(daily_data)
    except Exception as e:
        print(f"Error on {start_date.date()}: {e}")
    
    start_date += pd.Timedelta(days=1)
    time.sleep(1)  # เพื่อป้องกัน Rate Limit ของ API

# แปลงเป็น DataFrame และบันทึก CSV
df = pd.DataFrame(all_data)
df.to_csv("crypto_top_100_history.csv", index=False)
print("Data saved successfully.")


In [ ]:
all_data

In [ ]:
import requests
import pandas as pd
import time

# ฟังก์ชันดึง Top 100 Coins + Market Cap
def get_top_100(date):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1,
        "sparkline": "false",
        "price_change_percentage": "24h"
    }
    
    try:
        response = requests.get(url, params=params)
        data = response.json()

        # ตรวจสอบว่าได้รับข้อมูลในรูปแบบที่คาดไว้
        if isinstance(data, list):  # ตรวจสอบว่าเป็น list ของเหรียญ
            top_100 = [
                {
                    "date": date,
                    "id": coin["id"],
                    "symbol": coin["symbol"],
                    "name": coin["name"],
                    "market_cap": coin["market_cap"],
                    "price": coin["current_price"],
                    "rank": idx + 1
                }
                for idx, coin in enumerate(data)
            ]
            return top_100
        else:
            print(f"Error: Unexpected response format for {date}: {data}")
            return []

    except Exception as e:
        print(f"Error fetching {date}: {e}")
        return []

# ตั้งค่าช่วงเวลา
start_date = pd.to_datetime("2020-01-01")
end_date = pd.to_datetime("2025-03-05")

# เก็บข้อมูลทั้งหมด
all_data = []

while start_date <= end_date:
    print(f"Fetching data for {start_date.date()} ...")
    
    daily_data = get_top_100(start_date.date())
    if daily_data:
        all_data.extend(daily_data)
    
    start_date += pd.Timedelta(days=1)
    time.sleep(1.5)  # ป้องกัน API Rate Limit

# แปลงเป็น DataFrame และบันทึก CSV
df = pd.DataFrame(all_data)
df.to_csv("crypto_top_100_2020_2025.csv", index=False)
print("✅ Data saved successfully.")


### coinmaket

d5461d2c-a140-4932-b8bd-3a9e037afc58

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# ใส่ API Key ของคุณ
api_key = 'd5461d2c-a140-4932-b8bd-3a9e037afc58'

# ฟังก์ชันดึงข้อมูลจาก CoinMarketCap สำหรับราคาย้อนหลัง
def get_historical_data(coin_id, date):
    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical"
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    params = {
        'id': coin_id,
        'convert': 'USD',
        'time_start': date,  # วันที่ที่ต้องการดึงข้อมูล
        'time_end': date,
        'interval': 'daily'
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'data' in data:
            coin_data = data['data']['quotes'][0]
            return {
                "date": date,
                "id": coin_id,
                "price": coin_data["quote"]["USD"]["close"],  # ราคาปิด
                "market_cap": coin_data["quote"]["USD"]["market_cap"],  # Market Cap
            }
        else:
            print(f"Error: {data.get('status', {}).get('error_message', 'Unknown error')}")
            return None

    except Exception as e:
        print(f"Error fetching data for {coin_id} on {date}: {e}")
        return None

# ฟังก์ชันดึงข้อมูล Top 100 Coins
def get_top_100_from_coinmarketcap():
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    headers = {
        'X-CMC_PRO_API_KEY': api_key,
        'Accept': 'application/json'
    }
    params = {
        'limit': 100,
        'convert': 'USD'
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if 'data' in data:
            top_100 = [
                {
                    "id": coin["id"],
                    "name": coin["name"],
                    "symbol": coin["symbol"],
                    "rank": coin["cmc_rank"]
                }
                for coin in data['data']
            ]
            return top_100
        else:
            print(f"Error: {data.get('status', {}).get('error_message', 'Unknown error')}")
            return []

    except Exception as e:
        print(f"Error fetching Top 100 data: {e}")
        return []

# ตั้งค่าช่วงเวลา
start_date = datetime(2025, 3, 1)
end_date = datetime(2025, 3, 5)

# เก็บข้อมูลทั้งหมด
all_data = []

# ดึงข้อมูล Top 100 Coins
top_100_coins = get_top_100_from_coinmarketcap()

# Loop ผ่านแต่ละวัน
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%Y-%m-%d')  # เปลี่ยนวันที่เป็นรูปแบบ string
    print(f"Fetching data for {date_str} ...")

    # Loop ผ่านแต่ละเหรียญใน Top 100
    for coin in top_100_coins:
        coin_data = get_historical_data(coin['id'], date_str)
        if coin_data:
            all_data.append(coin_data)

    current_date += timedelta(days=1)  # เพิ่มวันถัดไป
    time.sleep(1.5)  # หน่วงเวลา 1.5 วินาทีเพื่อป้องกัน Rate Limit

# แปลงเป็น DataFrame และบันทึก CSV
if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv("coinmarketcap_top_100_2020_2025.csv", index=False)
    print("✅ Data saved successfully.")
else:
    print("No data available.")


### yf

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# กำหนด API ของ CoinGecko
base_url = "https://api.coingecko.com/api/v3/"

# ฟังก์ชันดึงข้อมูลเหรียญ Top 100 ในวันที่กำหนด
def fetch_top_100_coins(date):
    url = f"{base_url}coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1,
        "date": date  # วันที่ต้องการ
    }
    response = requests.get(url, params=params)
    
    try:
        data = response.json()
        # ตรวจสอบว่า data เป็น list หรือไม่
        if isinstance(data, list):
            print(f"Data fetched for {date}: {data[:5]}")  # ดูข้อมูลบางส่วน
            return data
        else:
            print(f"Unexpected data format for {date}: {data}")
            return []
    except Exception as e:
        print(f"Error parsing data for {date}: {e}")
        return []

# ดึงข้อมูล Top 100 Coins ตั้งแต่ 2020-01-01 ถึง 2025-03-05
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 5)

all_top_100 = []

# คำนวณช่วงเวลา 1 วัน
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%d-%m-%Y")
    print(f"Fetching data for {date_str}...")
    
    # ดึงข้อมูล Top 100 ในวันที่กำหนด
    top_100 = fetch_top_100_coins(date_str)
    
    # ตรวจสอบว่ามีข้อมูลหรือไม่
    if top_100:
        for coin in top_100:
            if isinstance(coin, dict):
                all_top_100.append({
                    "date": date_str,
                    "symbol": coin.get("symbol", ""),
                    "name": coin.get("name", ""),
                    "market_cap": coin.get("market_cap", 0),
                    "current_price": coin.get("current_price", 0),
                })
    
    # เลื่อนไปยังวันถัดไป
    current_date += timedelta(days=1)

# สร้าง DataFrame เพื่อจัดเก็บข้อมูลทั้งหมด
df_top_100 = pd.DataFrame(all_top_100)

# บันทึกข้อมูลลงใน CSV
df_top_100.to_csv("top_100_coins_2020_2025.csv", index=False)
print("✅ Data saved successfully.")


In [ ]:
all_top_100

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# กำหนด API ของ CoinGecko
base_url = "https://api.coingecko.com/api/v3/"

# ฟังก์ชันดึงข้อมูลเหรียญ Top 100 ในวันที่กำหนด
def fetch_top_100_coins(date):
    url = f"{base_url}coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1,
        "date": date  # วันที่ต้องการ
    }
    response = requests.get(url, params=params)
    
    try:
        data = response.json()
        if isinstance(data, list):
            return data
        else:
            print(f"Unexpected data format for {date}: {data}")
            return []
    except Exception as e:
        print(f"Error parsing data for {date}: {e}")
        return []

# ฟังก์ชันดึงราคาของเหรียญในวันที่กำหนด
def fetch_historical_price(coin_id, date):
    url = f"{base_url}coins/{coin_id}/history"
    params = {
        "date": date,  # วันที่ต้องการ
        "localization": "false"  # ไม่ต้องการ localization
    }
    response = requests.get(url, params=params)
    
    try:
        data = response.json()
        if "market_data" in data:
            price = data["market_data"]["current_price"]["usd"]
            return price
        else:
            return 0
    except Exception as e:
        print(f"Error fetching historical price for {coin_id} on {date}: {e}")
        return 0

# ดึงข้อมูล Top 100 Coins ตั้งแต่ 2020-01-01 ถึง 2025-03-05
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 3, 5)

all_top_100 = []

# คำนวณช่วงเวลา 1 วัน
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%d-%m-%Y")
    print(f"Fetching data for {date_str}...")
    
    # ดึงข้อมูล Top 100 ในวันที่กำหนด
    top_100 = fetch_top_100_coins(date_str)
    
    # ตรวจสอบว่ามีข้อมูลหรือไม่
    if top_100:
        for coin in top_100:
            if isinstance(coin, dict):
                # ดึงราคาของเหรียญในวันนั้นๆ
                coin_id = coin.get("id", "")
                price_on_date = fetch_historical_price(coin_id, date_str)
                
                all_top_100.append({
                    "date": date_str,
                    "symbol": coin.get("symbol", ""),
                    "name": coin.get("name", ""),
                    "market_cap": coin.get("market_cap", 0),
                    "current_price": price_on_date,
                })
    
    # เลื่อนไปยังวันถัดไป
    current_date += timedelta(days=1)

# สร้าง DataFrame เพื่อจัดเก็บข้อมูลทั้งหมด
df_top_100 = pd.DataFrame(all_top_100)

# บันทึกข้อมูลลงใน CSV
df_top_100.to_csv("top_100_coins_2020_2025.csv", index=False)
print("✅ Data saved successfully.")


In [ ]:
all_top_100

In [ ]:
import requests
from datetime import datetime, timedelta

# กำหนด URL ของ CoinGecko API
base_url = "https://api.coingecko.com/api/v3/"

# ฟังก์ชันดึงข้อมูล market cap ของเหรียญในวันที่กำหนด
def fetch_market_cap_on_date(coin_id, date):
    url = f"{base_url}coins/{coin_id}/history"
    params = {
        "date": date,
        "localization": "false"  # ปิด localization เพื่อให้ค่าเป็น USD
    }
    response = requests.get(url, params=params)
    
    try:
        data = response.json()
        if "market_data" in data:
            market_cap = data["market_data"]["market_cap"]["usd"]
            return market_cap
        else:
            return 0
    except Exception as e:
        print(f"Error fetching market cap for {coin_id} on {date}: {e}")
        return 0

# ตัวอย่างการดึงข้อมูล market cap สำหรับ Bitcoin ในวันที่ 2020-01-01
coin_id = "iota"  # ตัวอย่างเหรียญ Bitcoin
date = "01-01-2025"
market_cap = fetch_market_cap_on_date(coin_id, date)
print(f"Market Cap of {coin_id} on {date}: {market_cap} USD")


In [ ]:
import requests

def fetch_circulating_supply(coin_id='bitcoin'):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}'
    response = requests.get(url)
    data = response.json()
    return data['market_data']['circulating_supply']

# ตัวอย่างการดึง circulating supply ของ Bitcoin
circulating_supply_btc = fetch_circulating_supply('bitcoin')
print(f'Circulating Supply of Bitcoin: {circulating_supply_btc}')


In [ ]:
import requests
from datetime import datetime

# ฟังก์ชันดึง circulating supply ของเหรียญจาก CoinGecko API ณ วันที่กำหนด
def fetch_circulating_supply_on_date(coin_id='bitcoin', date='2025-03-06'):
    # เปลี่ยนวันที่จาก YYYY-MM-DD เป็น DD-MM-YYYY
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    formatted_date = date_obj.strftime("%d-%m-%Y")  # เปลี่ยนเป็นรูปแบบ DD-MM-YYYY
    
    # URL สำหรับดึงข้อมูล historical data
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/history'
    params = {
        'date': formatted_date,  # วันที่ที่ต้องการ (ในรูปแบบ DD-MM-YYYY)
        'localization': 'false',  # ไม่ต้องการข้อมูล localization
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # ตรวจสอบข้อมูลที่ได้
        if 'market_data' in data:
            circulating_supply = data['market_data'].get('circulating_supply', 'N/A')
            return circulating_supply
        else:
            return "Data not available for this date"
    else:
        return f"Error fetching data: {response.status_code}"

# ตัวอย่างการดึง circulating supply ของ Bitcoin ณ วันที่ 2025-01-01
circulating_supply_btc = fetch_circulating_supply_on_date('bitcoin', '2025-03-06')
print(f'Circulating Supply of Bitcoin on 2025-03-06: {circulating_supply_btc}')


In [ ]:
import requests
import time
import csv
from datetime import datetime

# ฟังก์ชันดึงข้อมูล Top 100 Coins จาก CoinGecko
def fetch_top_100_coins():
    url = 'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',  # เรียงลำดับตาม market cap
        'per_page': 100,  # ดึง 100 เหรียญ
        'page': 1,
    }
    response = requests.get(url, params=params)
    return response.json()

# ฟังก์ชันดึงข้อมูลราคาและ market cap ของเหรียญในวันที่กำหนด
def fetch_historical_data(coin_id, date):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/history'
    params = {
        'date': date,  # วันที่ต้องการในรูปแบบ DD-MM-YYYY
        'localization': 'false',  # ไม่ต้องการข้อมูล localization
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'market_data' in data:
            market_cap = data['market_data'].get('market_cap', {}).get('usd', 0)
            current_price = data['market_data'].get('current_price', {}).get('usd', 0)
            return current_price, market_cap
        else:
            return None, None
    else:
        return None, None

# ฟังก์ชันหลักในการดึงข้อมูลและบันทึกลงไฟล์
def fetch_and_save_data(start_date, end_date):
    # เปลี่ยนวันที่ให้เป็น object datetime
    start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')

    # สร้างไฟล์ CSV
    with open('top_100_coins_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['date', 'coin_name', 'coin_symbol', 'price_usd', 'market_cap_usd'])

        # ดึงข้อมูลเหรียญ Top 100
        top_100_coins = fetch_top_100_coins()

        # ดึงข้อมูลในช่วงวันที่ที่กำหนด
        current_date = start_date_obj
        while current_date <= end_date_obj:
            date_str = current_date.strftime('%d-%m-%Y')
            for coin in top_100_coins:
                coin_id = coin['id']
                coin_name = coin['name']
                coin_symbol = coin['symbol']

                # ดึงข้อมูลราคาและ market cap ของเหรียญในวันนั้น
                price, market_cap = fetch_historical_data(coin_id, date_str)

                if price is not None and market_cap is not None:
                    writer.writerow([date_str, coin_name, coin_symbol, price, market_cap])

            current_date += timedelta(days=1)
            time.sleep(1)  # ใส่การหน่วงเวลาเพื่อไม่ให้เกิน rate limit ของ API

# เรียกใช้งานฟังก์ชัน
fetch_and_save_data('2025-03-01', '2025-03-05')


In [ ]:
import requests
import pandas as pd

# ฟังก์ชันดึงข้อมูล Top 100 Coins ของวันนั้น ๆ
def fetch_top_100_coins(date_str):
    url = f'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'date': date_str
    }
    response = requests.get(url, params=params)
    data = response.json()
    if isinstance(data, list):
        print(f"Fetched {len(data)} coins for {date_str}")
        return data
    else:
        print(f"Error fetching data for {date_str}")
        return []

# สร้างลิสต์วันที่ตั้งแต่ 2025-01-01 ถึง 2025-03-05
dates = pd.date_range(start='2025-01-01', end='2025-03-05').strftime('%Y-%m-%d').tolist()

# ตัวแปรเก็บข้อมูลเหรียญทั้งหมด
coins_data = []

# ดึงข้อมูลแต่ละวัน
for date in dates:
    coins = fetch_top_100_coins(date)
    for coin in coins:
        coins_data.append({
            "date": date,
            "coin_name": coin.get("name", ""),
            "coin_symbol": coin.get("symbol", ""),
            "price_usd": coin.get("current_price", 0),
            "market_cap_usd": coin.get("market_cap", 0)
        })

# สร้าง DataFrame และบันทึกข้อมูล
df = pd.DataFrame(coins_data)
df.to_csv('top_100_coins_2025.csv', index=False)

# แสดงข้อความว่าบันทึกสำเร็จ
print("Data saved to 'top_100_coins_2025.csv'")


In [ ]:
import requests

def fetch_top_100_coins(date_str):
    url = f'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'date': date_str
    }
    response = requests.get(url, params=params)
    data = response.json()
    if isinstance(data, list):
        print(f"Fetched {len(data)} coins for {date_str}")
        return data
    else:
        print(f"Error fetching data for {date_str}")
        return []

# ตรวจสอบจำนวนเหรียญในแต่ละวัน
dates = ['2020-01-01', '2020-01-02', '2020-01-03']
for date in dates:
    coins = fetch_top_100_coins(date)
    if len(coins) < 100:
        print(f"Warning: Less than 100 coins available on {date}")
    else:
        print(f"Successfully fetched 100 coins on {date}")


In [ ]:
import pandas as pd

# ตัวอย่างข้อมูลที่ดึงมา
coins_data = []
dates = ['2025-03-02', '2025-03-03', '2025-03-04']

# ฟังก์ชันดึงข้อมูล
def fetch_top_100_coins(date_str):
    url = f'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'date': date_str  # เพิ่มวันที่
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

for date in dates:
    coins = fetch_top_100_coins(date)
    
    # ดีบักแสดงผลข้อมูลแต่ละวัน
    print(f"Fetching data for {date}...")
    print(f"Fetched {len(coins)} coins")
    
    for coin in coins:
        print(f"Coin: {coin['name']} - Price: {coin['current_price']}, Market Cap: {coin['market_cap']}")  # ดูข้อมูลที่ได้
        coins_data.append({
            "date": date,
            "coin_name": coin.get("name", ""),
            "coin_symbol": coin.get("symbol", ""),
            "price_usd": coin.get("current_price", 0),
            "market_cap_usd": coin.get("market_cap", 0)
        })

# สร้าง DataFrame
df = pd.DataFrame(coins_data)

# บันทึกข้อมูลลงในไฟล์ CSV
df.to_csv('top_100_coins_2025_03.csv', index=False)

# ตรวจสอบการบันทึกข้อมูล
print("Data saved to 'top_100_coins_2025_03.csv'")


In [ ]:
df 

In [ ]:
import requests
from datetime import datetime

def fetch_historical_data(coin_id='Bitcoin', date='2025-03-06'):
    # Convert date to timestamp format
    date_obj = datetime.strptime(date, '%Y-%m-%d')
    timestamp = int(date_obj.timestamp())
    
    # Make request to CoinGecko API
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 'max',  # Get max history
        'interval': 'daily'
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    # ตรวจสอบว่า 'prices' มีอยู่ในข้อมูลหรือไม่
    if 'prices' not in data:
        return f"Error: 'prices' not found in data for {coin_id} on {date}"
    
    # Find the data for the given date
    for i in range(len(data['prices'])):
        timestamp_in_data = data['prices'][i][0] / 1000  # Convert to seconds
        if timestamp_in_data == timestamp:
            price = data['prices'][i][1]  # Get the price at that time
            return price
        
    return "No data for this date"

# Example of fetching the closing price of Bitcoin for the given date
price = fetch_historical_data('Bitcoin', '2025-03-06')
print(f"The closing price of Bitcoin on 2025-03-06 is: {price}")



In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# ฟังก์ชันดึงข้อมูลย้อนหลังจาก CoinGecko
def fetch_historical_coin_data(coin_id, date_str):
    formatted_date = datetime.strptime(date_str, "%Y-%m-%d").strftime("%d-%m-%Y")  # แปลงเป็น "dd-mm-yyyy"
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/history"
    params = {
        "date": formatted_date,
        "localization": "false"
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if "market_data" in data:
            return {
                "date": date_str,
                "coin_name": data.get("name", ""),
                "coin_symbol": data.get("symbol", ""),
                "price_usd": data["market_data"].get("current_price", {}).get("usd", 0),
                "market_cap_usd": data["market_data"].get("market_cap", {}).get("usd", 0)
            }
    return None

# สร้างช่วงวันที่ที่ต้องการดึงข้อมูล
start_date = datetime(2025, 2, 26)
end_date = datetime(2025, 2, 28)

dates = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_date - start_date).days + 1)]

coins_data = []
top_5_coins = ["bitcoin", "ethereum", "tether", "bnb", "xrp"]  # ตัวอย่าง 5 เหรียญ

for date in dates:
    print(f"Fetching data for {date}...")
    for coin in top_5_coins:
        coin_data = fetch_historical_coin_data(coin, date)
        if coin_data:
            coins_data.append(coin_data)

# แปลงเป็น DataFrame และบันทึก CSV
df = pd.DataFrame(coins_data)
df.to_csv("historical_top_coins_2025_03.csv", index=False)

print("Data saved to 'historical_top_coins_2025_03.csv'")


In [ ]:
coins_data

In [ ]:
import requests

def fetch_historical_coin_data(coin_id, date_str):
    formatted_date = date_str.split("-")[2] + "-" + date_str.split("-")[1] + "-" + date_str.split("-")[0]  # "dd-mm-yyyy"
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/history"
    params = {"date": formatted_date, "localization": "false"}
    response = requests.get(url, params=params)
    return response.json()  # ดูข้อมูลที่ API คืนมา

date = "2025-03-01"
coin = "bitcoin"
data = fetch_historical_coin_data(coin, date)
print(data)  # เช็คว่ามี market_data หรือไม่


In [ ]:
data

In [ ]:
import requests
import pandas as pd

# URL ของไฟล์ CSV ที่มีอยู่แล้ว
url = "https://www.coingecko.com/en/coins/ethereum/historical_data/price_charts/export/ethereum/usd.csv"
# url ="/price_charts/export/ethereum/usd.csv"
# ดาวน์โหลดไฟล์ CSV
response = requests.get(url)

if response.status_code == 200:
    with open("ethereum_data.csv", "wb") as f:
        f.write(response.content)
    
    # โหลดไฟล์ CSV เข้าสู่ DataFrame
    df = pd.read_csv("ethereum_data.csv")
    
    # แสดงข้อมูล
    print(df.head())
else:
    print("Error: ไม่สามารถดาวน์โหลดข้อมูลได้")


In [ ]:
pip install selenium webdriver-manager


### อันนี้ใช้ได้


In [9]:
import requests

# ดึงข้อมูลเหรียญคริปโตทั้งหมดจาก CoinGecko
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 200,
    'page': 1
}
response = requests.get(url, params=params)
data = response.json()

# สร้าง list ของชื่อเหรียญ
crypto_names = [coin['name'].lower() for coin in data]

# แสดงผลลัพธ์
print(crypto_names)


['bitcoin', 'ethereum', 'xrp', 'tether', 'bnb', 'solana', 'usdc', 'cardano', 'dogecoin', 'lido staked ether', 'tron', 'pi network', 'wrapped bitcoin', 'chainlink', 'hedera', 'wrapped steth', 'stellar', 'sui', 'leo token', 'avalanche', 'usds', 'shiba inu', 'litecoin', 'bitcoin cash', 'toncoin', 'mantra', 'polkadot', 'weth', 'bitget token', 'ethena usde', 'hyperliquid', 'wrapped eeth', 'uniswap', 'whitebit coin', 'monero', 'near protocol', 'ondo', 'aptos', 'dai', 'susds', 'aave', 'ethereum classic', 'internet computer', 'pepe', 'official trump', 'gate', 'coinbase wrapped btc', 'okb', 'mantle', 'cronos', 'bittensor', 'tokenize xchange', 'vechain', 'pol (ex-matic)', 'first digital usd', 'ethena', 'algorand', 'render', 'filecoin', 'kaspa', 'cosmos hub', 'lombard staked btc', 'arbitrum', 'sonic (prev. ftm)', 'celestia', 'fasttoken', 'jupiter', 'artificial superintelligence alliance', 'optimism', 'kucoin', 'binance-peg weth', 'solv protocol solvbtc', 'story', 'kelp dao restaked eth', 'quant',

In [1]:
import requests

# ดึงข้อมูลเหรียญคริปโตทั้งหมดจาก CoinGecko
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 200,
    'page': 1
}
response = requests.get(url, params=params)
data = response.json()

# สร้าง list ของชื่อเหรียญ โดยแทนที่ช่องว่างด้วย "-"
crypto_names = [coin['name'].lower().replace(" ", "-") for coin in data]

# แสดงผลลัพธ์
print(crypto_names)


['bitcoin', 'ethereum', 'xrp', 'tether', 'bnb', 'solana', 'usdc', 'cardano', 'dogecoin', 'tron', 'lido-staked-ether', 'pi-network', 'wrapped-bitcoin', 'chainlink', 'hedera', 'leo-token', 'wrapped-steth', 'stellar', 'sui', 'avalanche', 'usds', 'bitcoin-cash', 'shiba-inu', 'litecoin', 'toncoin', 'polkadot', 'mantra', 'weth', 'bitget-token', 'ethena-usde', 'hyperliquid', 'wrapped-eeth', 'uniswap', 'whitebit-coin', 'monero', 'aptos', 'near-protocol', 'ondo', 'dai', 'susds', 'aave', 'ethereum-classic', 'internet-computer', 'pepe', 'gate', 'okb', 'official-trump', 'coinbase-wrapped-btc', 'mantle', 'tokenize-xchange', 'cronos', 'bittensor', 'vechain', 'pol-(ex-matic)', 'first-digital-usd', 'algorand', 'ethena', 'filecoin', 'render', 'kaspa', 'cosmos-hub', 'lombard-staked-btc', 'arbitrum', 'fasttoken', 'celestia', 'sonic-(prev.-ftm)', 'artificial-superintelligence-alliance', 'jupiter', 'optimism', 'kucoin', 'solv-protocol-solvbtc', 'binance-peg-weth', 'story', 'kelp-dao-restaked-eth', 'movemen

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from webdriver_manager.chrome import ChromeDriverManager

# ตั้งค่า Chrome ให้ดาวน์โหลดไฟล์อัตโนมัติ
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": ".",  # โฟลเดอร์ที่ต้องการบันทึกไฟล์
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})

# เปิด WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# เปิดหน้า CoinGecko
url = "https://www.coingecko.com/en/coins/bitcoin/historical_data"
driver.get(url)

try:
    wait = WebDriverWait(driver, 10)  # รอสูงสุด 10 วินาที

    # ✅ คลิกปุ่มดาวน์โหลด (ไอคอนลูกศรลง)
    download_button = wait.until(EC.element_to_be_clickable((By.ID, "export")))
    download_button.click()
    print("📥 กดปุ่มดาวน์โหลดแล้ว!")

    # ✅ หา data-url ของ .csv
    csv_button = wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), '.csv')]")))
    csv_url = csv_button.get_attribute("data-url")

    if csv_url:
        full_url = "https://www.coingecko.com" + csv_url  # เติม URL เต็ม
        print(f"📥 ดาวน์โหลด CSV จาก: {full_url}")
        driver.get(full_url)  # เปิดลิงก์โดยตรง

    # ✅ รอให้ไฟล์โหลดเสร็จ
    time.sleep(10)

except Exception as e:
    print("❌ Error:", e)

# ปิดเบราว์เซอร์
driver.quit()


In [5]:
len(crypto_names)

200

whitebit
near
gatetoken
pol-ex-matic
sonic
solv-protocol-solvbtc
wbnb
arbitrum-bridged-wbtc-arbitrum-one
binance-bridged-usdc-bnb-smart-chain
curve-dao-token
l2-standard-bridged-weth-base
true-usd
virtual-protocol
arbitrum-bridged-weth-arbitrum-one
avalanche-bridged-btc-avalanche
ether-fi-staked-eth
bridged-usdc-polygon-pos-bridge
verus-coin
polygon-pos-bridged-weth-polygon-pos
mantle-bridged-usdt-mantle


In [9]:
crypto_names

['bitcoin',
 'ethereum',
 'xrp',
 'tether',
 'bnb',
 'solana',
 'usdc',
 'cardano',
 'dogecoin',
 'tron',
 'lido-staked-ether',
 'pi-network',
 'wrapped-bitcoin',
 'chainlink',
 'hedera',
 'leo-token',
 'wrapped-steth',
 'stellar',
 'sui',
 'avalanche',
 'usds',
 'bitcoin-cash',
 'shiba-inu',
 'litecoin',
 'toncoin',
 'polkadot',
 'mantra',
 'weth',
 'bitget-token',
 'ethena-usde',
 'hyperliquid',
 'wrapped-eeth',
 'uniswap',
 'whitebit-coin',
 'monero',
 'aptos',
 'near-protocol',
 'ondo',
 'dai',
 'susds',
 'aave',
 'ethereum-classic',
 'internet-computer',
 'pepe',
 'gate',
 'okb',
 'official-trump',
 'coinbase-wrapped-btc',
 'mantle',
 'tokenize-xchange',
 'cronos',
 'bittensor',
 'vechain',
 'pol-(ex-matic)',
 'first-digital-usd',
 'algorand',
 'ethena',
 'filecoin',
 'render',
 'kaspa',
 'cosmos-hub',
 'lombard-staked-btc',
 'arbitrum',
 'fasttoken',
 'celestia',
 'sonic-(prev.-ftm)',
 'artificial-superintelligence-alliance',
 'jupiter',
 'optimism',
 'kucoin',
 'solv-protocol-so

In [10]:
crypto_list = [
    "whitebit",
    "near",
    "gatetoken",
    "pol-ex-matic",
    "sonic",
    "solv-protocol-solvbtc",
    "wbnb",
    "arbitrum-bridged-wbtc-arbitrum-one",
    "binance-bridged-usdc-bnb-smart-chain",
    "curve-dao-token",
    "l2-standard-bridged-weth-base",
    "true-usd",
    "virtual-protocol",
    "arbitrum-bridged-weth-arbitrum-one",
    "avalanche-bridged-btc-avalanche",
    "ether-fi-staked-eth",
    "bridged-usdc-polygon-pos-bridge",
    "verus-coin",
    "polygon-pos-bridged-weth-polygon-pos",
    "mantle-bridged-usdt-mantle"
]

# แสดงผลข้อมูล
print(crypto_list)


['whitebit', 'near', 'gatetoken', 'pol-ex-matic', 'sonic', 'solv-protocol-solvbtc', 'wbnb', 'arbitrum-bridged-wbtc-arbitrum-one', 'binance-bridged-usdc-bnb-smart-chain', 'curve-dao-token', 'l2-standard-bridged-weth-base', 'true-usd', 'virtual-protocol', 'arbitrum-bridged-weth-arbitrum-one', 'avalanche-bridged-btc-avalanche', 'ether-fi-staked-eth', 'bridged-usdc-polygon-pos-bridge', 'verus-coin', 'polygon-pos-bridged-weth-polygon-pos', 'mantle-bridged-usdt-mantle']


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from webdriver_manager.chrome import ChromeDriverManager



# ตั้งค่า Chrome ให้ดาวน์โหลดไฟล์อัตโนมัติ
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": ".",  # โฟลเดอร์ที่ต้องการบันทึกไฟล์
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})

# เปิด WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

for coin in crypto_list:
    # เปลี่ยน URL ให้ตรงกับเหรียญแต่ละตัว
    url = f"https://www.coingecko.com/en/coins/{coin}/historical_data"
    driver.get(url)
    print(f"🔎 เปิดหน้า {coin} แล้ว")
    
    try:
        wait = WebDriverWait(driver, 10)  # รอสูงสุด 10 วินาที

        # คลิกปุ่มดาวน์โหลด (ไอคอนลูกศรลง)
        download_button = wait.until(EC.element_to_be_clickable((By.ID, "export")))
        download_button.click()
        print(f"📥 กดปุ่มดาวน์โหลดสำหรับ {coin} แล้ว!")

        # หา element ที่มี .csv จาก attribute data-url
        csv_button = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//span[contains(text(), '.csv')]")
        ))
        csv_url = csv_button.get_attribute("data-url")

        if csv_url:
            full_url = "https://www.coingecko.com" + csv_url  # เติม URL เต็ม
            print(f"📥 ดาวน์โหลด CSV สำหรับ {coin} จาก: {full_url}")
            driver.get(full_url)  # เปิดลิงก์โดยตรงเพื่อดาวน์โหลดไฟล์

        # รอให้ดาวน์โหลดเสร็จ (ปรับเวลาได้ตามความเหมาะสม)
        time.sleep(10)

    except Exception as e:
        print(f"❌ Error สำหรับ {coin}: {e}")

# ปิดเบราว์เซอร์เมื่อทำงานเสร็จ
driver.quit()


🔎 เปิดหน้า whitebit แล้ว
📥 กดปุ่มดาวน์โหลดสำหรับ whitebit แล้ว!
📥 ดาวน์โหลด CSV สำหรับ whitebit จาก: https://www.coingecko.com/price_charts/export/whitebit/usd.csv
🔎 เปิดหน้า near แล้ว
📥 กดปุ่มดาวน์โหลดสำหรับ near แล้ว!
📥 ดาวน์โหลด CSV สำหรับ near จาก: https://www.coingecko.com/price_charts/export/near/usd.csv
🔎 เปิดหน้า gatetoken แล้ว
📥 กดปุ่มดาวน์โหลดสำหรับ gatetoken แล้ว!
📥 ดาวน์โหลด CSV สำหรับ gatetoken จาก: https://www.coingecko.com/price_charts/export/gatetoken/usd.csv
🔎 เปิดหน้า pol-ex-matic แล้ว
📥 กดปุ่มดาวน์โหลดสำหรับ pol-ex-matic แล้ว!
📥 ดาวน์โหลด CSV สำหรับ pol-ex-matic จาก: https://www.coingecko.com/price_charts/export/pol-ex-matic/usd.csv
🔎 เปิดหน้า sonic แล้ว
📥 กดปุ่มดาวน์โหลดสำหรับ sonic แล้ว!
📥 ดาวน์โหลด CSV สำหรับ sonic จาก: https://www.coingecko.com/price_charts/export/sonic/usd.csv
🔎 เปิดหน้า solv-protocol-solvbtc แล้ว
📥 กดปุ่มดาวน์โหลดสำหรับ solv-protocol-solvbtc แล้ว!
📥 ดาวน์โหลด CSV สำหรับ solv-protocol-solvbtc จาก: https://www.coingecko.com/price_charts/export/so

### data cut

In [13]:
import pandas as pd
import glob
import os

# 📌 กำหนดช่วงวันที่ที่ต้องการ
start_date = "2020-01-01"
end_date = "2025-03-07"

# 📂 ค้นหาไฟล์ CSV ทั้งหมดในโฟลเดอร์ "coin/"
input_folder = "coin"
output_folder = "filtered_data"

# ✅ ตรวจสอบว่าโฟลเดอร์ "coin" มีอยู่จริง
if not os.path.exists(input_folder):
    print(f"❌ ไม่พบโฟลเดอร์ '{input_folder}'")
    exit()

# 📂 สร้างโฟลเดอร์ใหม่สำหรับเก็บไฟล์ที่ตัดข้อมูลแล้ว
os.makedirs(output_folder, exist_ok=True)

# 🔍 ดึงรายชื่อไฟล์ CSV ทั้งหมดในโฟลเดอร์ "coin/"
csv_files = glob.glob(os.path.join(input_folder, "*.csv"))

for file in csv_files:
    # 📖 อ่านไฟล์ CSV
    df = pd.read_csv(file)
    
    # 🛠 แปลงคอลัมน์วันที่ (snapped_at) เป็น datetime
    df["snapped_at"] = pd.to_datetime(df["snapped_at"], errors='coerce')
    
    # 🔍 ตรวจสอบช่วงวันที่
    min_date = df["snapped_at"].min().strftime("%Y-%m-%d")
    max_date = df["snapped_at"].max().strftime("%Y-%m-%d")
    
    print(f"📊 {file}: {min_date} -> {max_date}")  # แสดงช่วงวันที่ในไฟล์
    
    # ✂️ กรองข้อมูลให้อยู่ในช่วงที่ต้องการ
    df_filtered = df[(df["snapped_at"] >= start_date) & (df["snapped_at"] <= end_date)]
    
    # 💾 สร้างชื่อไฟล์ใหม่ในโฟลเดอร์ filtered_data/ โดยคงชื่อไฟล์เดิม
    output_file = os.path.join(output_folder, os.path.basename(file))
    df_filtered.to_csv(output_file, index=False)
    
    print(f"✅ บันทึกไฟล์ใหม่ที่: {output_file}")

print("🎉 เสร็จเรียบร้อย! ไฟล์ที่ถูกตัดข้อมูลจะอยู่ในโฟลเดอร์ 'filtered_data/'")


📊 coin\1inch-usd-max.csv: 2020-12-25 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\1inch-usd-max.csv
📊 coin\aave-usd-max.csv: 2020-10-03 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\aave-usd-max.csv
📊 coin\ada-usd-max.csv: 2017-10-18 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\ada-usd-max.csv
📊 coin\aero-usd-max.csv: 2023-09-15 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\aero-usd-max.csv
📊 coin\aioz-usd-max.csv: 2021-04-03 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\aioz-usd-max.csv
📊 coin\akt-usd-max.csv: 2020-10-16 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\akt-usd-max.csv
📊 coin\algo-usd-max.csv: 2019-06-21 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\algo-usd-max.csv
📊 coin\amp-usd-max.csv: 2020-09-09 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\amp-usd-max.csv
📊 coin\ape-usd-max.csv: 2022-03-17 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\ape-usd-max.csv
📊 coin\apt-usd-max.csv: 2022-10-19 -> 2025-03-07
✅ บันทึกไฟล์ใหม่ที่: filtered_data\apt-

In [14]:
import pandas as pd
import glob
import os

# 📂 โฟลเดอร์ที่เก็บไฟล์ CSV ที่ถูกกรองแล้ว
input_folder = "filtered_data"
output_folder = "top200_by_date"

# ✅ สร้างโฟลเดอร์ใหม่สำหรับเก็บผลลัพธ์
os.makedirs(output_folder, exist_ok=True)

# 🔍 ค้นหาไฟล์ CSV ทั้งหมดในโฟลเดอร์ filtered_data
csv_files = glob.glob(os.path.join(input_folder, "*.csv"))

# 📖 อ่านทุกไฟล์และรวมกันใน DataFrame เดียว
dataframes = []
for file in csv_files:
    df = pd.read_csv(file)

    # 📆 แปลงคอลัมน์วันที่
    df["snapped_at"] = pd.to_datetime(df["snapped_at"], errors='coerce')
    
    # 🔍 เพิ่มชื่อเหรียญ (จากชื่อไฟล์)
    coin_name = os.path.basename(file).replace(".csv", "")  # ตัด .csv ออก
    df["coin"] = coin_name  # เพิ่มคอลัมน์ 'coin'
    
    dataframes.append(df)

# 📊 รวมทุกเหรียญเป็น DataFrame เดียว
df_all = pd.concat(dataframes, ignore_index=True)

# 📆 ดึงวันที่ที่มีข้อมูลทั้งหมด (จากทุกเหรียญ)
unique_dates = sorted(df_all["snapped_at"].dropna().unique())

# 🔁 วนลูปทุกวันที่มีข้อมูล
for date in unique_dates:
    df_day = df_all[df_all["snapped_at"] == date].copy()

    # 🔢 เรียง market_cap จากมากไปน้อย
    df_day = df_day.sort_values(by="market_cap", ascending=False)

    # ✨ คัดเลือกแค่ Top 200
    df_day = df_day.head(200)

    # ✂️ คัดเฉพาะคอลัมน์ที่ต้องการ
    df_day = df_day[["snapped_at", "coin", "market_cap", "price", "total_volume"]]

    # 📌 ตั้งชื่อไฟล์ เช่น "2020-01-01_top200.csv"
    date_str = date.strftime("%Y-%m-%d")
    output_file = os.path.join(output_folder, f"{date_str}_top200.csv")

    # 💾 บันทึก CSV
    df_day.to_csv(output_file, index=False)

    print(f"✅ บันทึก: {output_file}")

print("🎉 เสร็จเรียบร้อย! ไฟล์ที่เรียงตาม market_cap ถูกบันทึกใน 'top200_by_date/'")


✅ บันทึก: top200_by_date\2020-01-01_top200.csv
✅ บันทึก: top200_by_date\2020-01-02_top200.csv
✅ บันทึก: top200_by_date\2020-01-03_top200.csv
✅ บันทึก: top200_by_date\2020-01-04_top200.csv
✅ บันทึก: top200_by_date\2020-01-05_top200.csv
✅ บันทึก: top200_by_date\2020-01-06_top200.csv
✅ บันทึก: top200_by_date\2020-01-07_top200.csv
✅ บันทึก: top200_by_date\2020-01-08_top200.csv
✅ บันทึก: top200_by_date\2020-01-09_top200.csv
✅ บันทึก: top200_by_date\2020-01-10_top200.csv
✅ บันทึก: top200_by_date\2020-01-11_top200.csv
✅ บันทึก: top200_by_date\2020-01-12_top200.csv
✅ บันทึก: top200_by_date\2020-01-13_top200.csv
✅ บันทึก: top200_by_date\2020-01-14_top200.csv
✅ บันทึก: top200_by_date\2020-01-15_top200.csv
✅ บันทึก: top200_by_date\2020-01-16_top200.csv
✅ บันทึก: top200_by_date\2020-01-17_top200.csv
✅ บันทึก: top200_by_date\2020-01-18_top200.csv
✅ บันทึก: top200_by_date\2020-01-19_top200.csv
✅ บันทึก: top200_by_date\2020-01-20_top200.csv
✅ บันทึก: top200_by_date\2020-01-21_top200.csv
✅ บันทึก: top